<a href="https://colab.research.google.com/github/bhattacharya5/MLBD/blob/main/M21AIE211_MLBD_Assignment2_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd "/content/gdrive/My Drive/Colab Notebooks/semester3/MLBD"

/content/gdrive/My Drive/Colab Notebooks/semester3/MLBD


In [3]:
import numpy as np
import pandas as pd

In [4]:
df_train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/semester3/MLBD/train.csv')
#df_train = pd.read_csv('/content/sample_data/train.csv')
df_train.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_train.reset_index(inplace=True)

In [6]:
df_test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/semester3/MLBD/test.csv')
#df_test = pd.read_csv('/content/sample_data/test.csv')
df_test.head(5)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df_test.reset_index(inplace=True)

In [8]:
Y_train = df_train["label"]

# Drop 'label' column
X_train = df_train.drop(labels = ["label"],axis = 1) 

X_test = df_test

In [ ]:
import numpy as np
import time
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# Create a generator function to load the data in chunks
def load_data_in_chunks(x_train, y_train, batch_size):
  for i in range(0, len(x_train), batch_size):
    yield x_train[i:i + batch_size], y_train[i:i + batch_size]

# Create a function to calculate the Jaccard similarity between two vectors
def jaccard_similarity(x, y):
  intersection = np.count_nonzero(x & y)
  union = np.count_nonzero(x | y)
  if union == 0:
    return np.nan
  else:
    return intersection / union

# Create a function to classify a test sample using the kNN algorithm and Jaccard similarity
def classify_sample(x_test_sample, x_train, y_train, k=5):
  # Check if y_train is empty
  if len(y_train) == 0:
    return None

  # Calculate the Jaccard similarity between the test sample and all of the training samples
  similarities = np.array([jaccard_similarity(x_test_sample, x_train_sample) for x_train_sample in x_train])

  # Find the k training samples with the highest Jaccard similarity to the test sample
  nearest_neighbors = np.argsort(similarities)[:k]

  # Get the labels of the k nearest neighbors
  nearest_neighbor_labels = y_train[nearest_neighbors]

  # Return the most common label among the k nearest neighbors
  return np.argmax(np.bincount(nearest_neighbor_labels))

# Create a function to classify all of the test samples using the kNN algorithm and Jaccard similarity
def classify_all_samples(x_test, x_train, y_train, k=5):
  # Initialize the predictions array
  predictions = np.zeros(len(x_test))

  # Classify each test sample
  for i in range(len(x_test)):   

    # Calculate the Jaccard similarity between the test sample and all of the training samples
    #similarities = np.array([jaccard_similarity(x_test[i], x_train_sample) for x_train_sample in x_train])
    similarities = []  
    
    x = x_test.iloc[i]   
    
    for y in x_train:
      intersection = np.count_nonzero(x & y)
      union = np.count_nonzero(x | y)
      if union == 0:
         similarities.append(np.nan)
      else:
        similarities.append(intersection / union)

    # Find the k training samples with the highest Jaccard similarity to the test sample
    nearest_neighbors = np.argsort(similarities)[:k]   
        
    # Get the labels of the k nearest neighbors
    nearest_neighbor_labels = Y_train[nearest_neighbors]   

    # Return the most common label among the k nearest neighbors
    predictions[i] =  np.argmax(np.bincount(nearest_neighbor_labels))


  # Return the predictions array
  return predictions

# Set the values of k
ks = [1, 2, 3, 4, 5]

# Initialize the accuracy array
accuracies = np.zeros(len(ks))

# Start the timer
start_time = time.time()

# Classify all of the test samples for each value of k
for i, k in enumerate(ks):
  for x_batch, y_batch in load_data_in_chunks(x_train, y_train, 128):
    predictions = classify_all_samples(x_batch, y_batch, k)
    accuracies[i] += np.mean(predictions == y_batch)

# Stop the timer
end_time = time.time()

# Print the classification accuracies and time required to classify all the test samples
for i, k in enumerate(ks):
  print('Classification accuracy for k = {}: {}'.format(k, accuracies[i] / len(x_test)))
  print('Time required to classify all the test samples for k = {}: {}'.format(k, end_time - start_time))
